In [9]:
from cfdmod.use_cases.hfpi import solver, handler, analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pathlib

analysis.set_plt_style()

### Report directory
base_base = pathlib.Path("/mnt/disk01/prd-eng/portfolio-eng/simulation_files/petra_nova_fluidmechanics/comparison/dynamic_analysis")
subproject_path = base_base / "subproj_main"
structure_path = base_base / "struct_original"
case_path = subproject_path / "out"
data_path = subproject_path / "forces/"

base_folder = pathlib.Path("/mnt/disk01/prd-eng/cases/petra_nova_fluidmechanics/engmod_case")
experimental_data_path = base_folder / "../comparison/force_by_floor/exp_csv/"



In [10]:
### building dimensions
H = 250.424
Bx = 69.14
By = 90.6
B = max(Bx, By)

dimensions = handler.DimensionSpecs(base=B, height=H)

In [11]:
modes_csv = structure_path / "modes.csv"

n_modes = 12
floors_csv = structure_path / "nodes.csv"
phi_csvs = [structure_path / f"phi_{i}.csv" for i in range(1, n_modes+1)]

struct_data = solver.HFPIStructuralData.build(
    modes_csv=modes_csv,
    floors_csv=floors_csv,
    phi_floors_csvs=phi_csvs,
    max_active_modes=5,
)

In [12]:
directions = ["000", "030", "060", "090", "120", "150", "180", "210", "240", "270", "300", "330"]
float_directions = [float(d) for d in directions]
recurrence_periods = [1, 10, 50]
xis = [0.01, 0.02]

In [13]:
directional_forces = {}

for d in directions:
    cf_x_h5 = data_path / f"FX_{d}.h5"
    cf_y_h5 = data_path / f"FY_{d}.h5"
    cm_z_h5 = data_path / f"MZ_{d}.h5"
    forces_data = solver.HFPIForcesData.build(cf_x_h5, cf_y_h5, cm_z_h5)
    directional_forces[float(d)] = forces_data

directional_forces.keys()

dict_keys([0.0, 30.0, 60.0, 90.0, 120.0, 150.0, 180.0, 210.0, 240.0, 270.0, 300.0, 330.0])

In [14]:
wind_analytical = handler.WindAnalysis(
    directional_velocity_multiplier={d: 1 for d in float_directions},
    directional_roughness_cats=pd.DataFrame(),
)

In [15]:
results_save = pathlib.Path("/mnt/dev01_fs_shared/waine/repos/cfdmod/output/hfpi_test")
analysis_full = handler.HFPIAnalysisHandler(
    wind_analytics=wind_analytical,
    dimensions=dimensions,
    structural_data=struct_data,
    directional_forces=directional_forces,
    save_folder=results_save,
)

In [16]:
all_parameters = analysis_full.generate_combined_parameters(
    directions=float_directions,
    recurrence_periods=recurrence_periods,
    xis=xis,
)


In [ ]:
handler.HFPIFullResults.load_all_results(results_folder=results_save)

{HFPICaseParameters(direction=0.0, xi=0.01, recurrence_period=1.0): HFPIResults(displacement={'x': array([[ 8.14032320e-05,  2.54324459e-04,  5.72112058e-04, ...,
          5.14708226e-02,  5.16809970e-02,  5.34142071e-02],
        [ 8.47424450e-05,  2.64813022e-04,  5.94987154e-04, ...,
          5.17212094e-02,  5.20358812e-02,  5.37568497e-02],
        [ 9.49870741e-05,  2.97177381e-04,  6.65323111e-04, ...,
          5.22180922e-02,  5.28478372e-02,  5.45353376e-02],
        ...,
        [ 4.52336998e-04,  1.38131670e-03,  3.06835622e-03, ...,
         -3.25033034e-02, -2.40331811e-02, -3.80348611e-02],
        [ 4.58432103e-04,  1.39406803e-03,  3.10466298e-03, ...,
         -4.31996512e-02, -3.54220439e-02, -5.13076060e-02],
        [ 4.64035557e-04,  1.40550484e-03,  3.13789410e-03, ...,
         -5.23915474e-02, -4.52552101e-02, -6.28728248e-02]],
       shape=(11251, 76)), 'y': array([[-2.28944604e-04, -6.77550056e-04, -1.47425865e-03, ...,
         -3.15778035e-01, -3.2035168